# Trace AI - AI4Indonesia (GovAI Hackathon)
---

**Trace.AI (Transparent Review and Cost Evaluation Powered by Gen-AI)** adalah suatu platform yang dapat melakukan pengecekan dokumen proposal pengadaan barang dan jasa secara otomatis membandingkan harga dengan data internal dan eksternal untuk mendeteksi indikasi mark-up (disesuaikan dengan PP Nomor 12 Tahun 2021 tentang pengadaan barang dan jasa).

Trace.AI memiliki 3 fitur utama sebagai berikut:

1. **Otomasi pengecekan dokumen proposal**: Gen AI dapat mengotomatisasi pengecekan seluruh dokumen pengadaan, termasuk proposal dari vendor, dengan membaca dan memahami konten secara cepat dimana sistem dapat memahami bahasa dalam proposal, kontrak, atau dokumen tender. 
  
2. **Deteksi Anomali/Mark-Up Harga**: Menggunakan machine learning untuk mendeteksi indikasi adanya mark-up harga berdasarkan pola data historis baik dari internal maupun external. Setiap kenaikan harga yang tidak wajar dapat diidentifikasi secara otomatis oleh algoritma, yang kemudian memberikan flagging, sehingga tim terkait dapat melakukan tindakan secara cepat.
  
3. **Rekomendasi berbasis data**: Gen AI dapat memberikan rekomendasi untuk memberikan rekomendasi harga yang wajar kepada pihak pengadaan, sehingga negosiasi dapat lebih efektif dan berdasarkan data yang lebih kuat sehingga dapat meminimasi kesalahan dan dapat mengambil langkah-langkah mitigasi risiko secara cepat dan tepat.

Diharapkan, dengan Trace.AI, dapat meningkatkan transparansi dan efektifitas proses pengadaan barang dan jasa sebesar 80%.

## 0. Permulaan
Bagian ini menjelaskan langkah-langkah untuk menginstal, melakukan autentikasi, dan inisialisasi penggunaan Google Cloud Vertex AI

In [ ]:
# Install Library
# pip install --upgrade google-cloud-aiplatform
# pip install PyPDF2
# pip install python-dotenv==1.0.1 # For reading environment variables stored in .env file
# pip install langchain==0.2.2
# pip install langchain-community==0.2.3
# pip install langchain-openai==0.1.8 # For embeddings
# pip install unstructured==0.14.4 # Document loading
# onnxruntime==1.17.1 # chromadb dependency: on Mac use `conda install onnxruntime -c conda-forge`
# For Windows users, install Microsoft Visual C++ Build Tools first
# install onnxruntime before installing `chromadb`
# pip install chromadb==0.5.0 # Vector storage
# pip install openai==1.31.1 # For embeddings
# pip install tiktoken==0.7.0  # For embeddings 

In [73]:
# Authentication to Google Cloud
# gcloud auth application-default login

In [23]:
# Import Libraries

import pandas as pd
import numpy as np

import json
import PyPDF2
import vertexai
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory, ChatSession

In [24]:
# Initiate Connection to GCP Vertex AI and Define Gemini Model

vertexai.init(project="tutorial-house", location="us-central1")

safety_settings = {
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH}

model_gemini = GenerativeModel(model_name="gemini-1.5-flash-002",
                               safety_settings=safety_settings)

## 1. Membaca Dokumen Pengadaan Barang dan Jasa (PDF)

Bagian ini menyajikan hasil ekstraksi teks dari file PDF yang berjudul **"Contoh Proposal Penawaran Pengadaan Alat Tulis Kantor"**. Proposal ini berisi rincian penawaran untuk pengadaan alat tulis kantor, mencakup informasi tentang produk, harga, dan layanan yang ditawarkan oleh penyedia. 

Teks yang dihasilkan pada bagian di bawah ini diambil langsung dari dokumen PDF dan disusun kembali untuk memudahkan pembacaan serta pemahaman isi proposal. Bagian ini dapat menjadi acuan dalam memahami isi lengkap dari proposal pengadaan yang diajukan.


In [25]:
# Path to your PDF file
file_path = "Contoh Proposal Penawaran Pengadaan Alat Tulis Kantor.pdf"

# Read and extract text from PDF
document_text = ""
with open(file_path, "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        document_text += page.extract_text() + "\n"

print(document_text)

Proposal Penawaran Pengadaan Alat Tulis Kantor (ATK)  
Diajukan oleh PT. Alat Kantor Nusantara  
Untuk Kementerian Keuangan
 
1. Latar Belakang  
Sebagai penyedia perlengkapan kantor terkemuka, PT. Alat Kantor Nusantara siap 
memenuhi kebutuhan ATK untuk mendukung operasional di Kementerian Keuangan. 
Kami menyediakan produk berkualitas tinggi dengan harga yang bersaing, pengiriman 
tepat waktu, serta dukungan layanan purna jual.  
 
2. Penawaran Produk  
Kami menawarkan pengadaan berbagai jenis ATK sebagai berikut:  
No. Nama Barang  Satuan  Jumlah  Harga Satuan 
(Rp)  Total Harga (Rp)  
1 Pulpen Gel  Pcs 2000  6,000,00  12,000,000,00  
2 Pensil Mekanik  Pcs 1000  6,000,00  6,000,000,00  
3 Kertas HVS A4 (80gsm)  Rim 400  41,000,00  16,400,000,00  
4 Kertas HVS A3 (70gsm)  Rim 150  70,000,00  10,500,000,00  
5 Penggaris Logam 30 
cm Pcs 200  4,250,00  850,000,00  
6 Amplop Coklat A4  Pack  150  30,000,00  4,500,000,00  
7 Map Snelhekter Plastik  Pack  300  11,000,00  3,300,000,00  
8 

## 2. Mengeluarkan Ringkasan Dokumen dengan AI

Agen AI telah kami rancang untuk menghasilkan ringkasan otomatis dari dokumen yang diinput. Dengan menyesuaikan konfigurasi model, agen ini dapat memberikan ringkasan yang akurat dan relevan, memungkinkan pembaca mendapatkan pemahaman cepat mengenai isi dokumen. Ringkasan yang dihasilkan disesuaikan berdasarkan prompt yang diformulasikan secara khusus dan output yang diatur untuk menjaga kualitas dan detail penting.

In [26]:
# Design AI Agent to Generate List of Items based on the Inputted Documents

prompt_summary = """Keluarkan summary dari document tersebut"""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}

response_summary = model_gemini.generate_content(
    [document_text, prompt_summary],
    generation_config=generation_config,
)

response_summary.text

/Users/ardaputra/Library/Python/3.9/lib/python/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


'PT. Alat Kantor Nusantara mengajukan penawaran pengadaan ATK kepada Kementerian Keuangan senilai Rp 160.727.500.  Penawaran mencakup 30 jenis ATK dengan jumlah dan harga satuan yang tercantum dalam proposal.  Layanan tambahan meliputi garansi produk 6 bulan, pengiriman maksimal 3 hari kerja, dan layanan purna jual.  PT. Alat Kantor Nusantara berkomitmen menyediakan produk berkualitas tinggi dengan harga kompetitif dan layanan optimal.\n'

## 3. Mengeluarkan list Barang dari Dokumen secara Otomatis dengan AI

Agen AI telah kami rancang untuk mengekstrak informasi spesifik mengenai barang dan jasa pengadaan yang tercantum dalam dokumen. Dengan menggunakan prompt dan konfigurasi yang dirancang khusus, agen AI menghasilkan daftar dalam format tabel JSON yang memuat rincian nama barang, jumlah, harga satuan, dan total harga.

In [35]:
# Design AI Agent to Generate List of Items based on the Inputted Documents

prompt_items = """Keluarkan list barang dan jasa pengadaan dari dokumen tersebut dalam bentuk table dengan format sebagai berikut; nama barang, jumlah, harga satuan, harga total. Keluarkan dalam bentuk json dengan nama 'data'"""

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
    "response_mime_type": "application/json"
}

response_item = model_gemini.generate_content(
    [document_text, prompt_items],
    generation_config=generation_config,
)

# Parse JSON string
parsed_data = json.loads(response_item.text)

# Convert data to DataFrame
item_df = pd.DataFrame(parsed_data["data"])

# Display the DataFrame
item_df

,nama_barang,jumlah,harga_satuan,harga_total
0,Pulpen Gel,2000,6000,12000000
1,Pensil Mekanik,1000,6000,6000000
2,Kertas HVS A4 (80gsm),400,41000,16400000
3,Kertas HVS A3 (70gsm),150,70000,10500000
4,Penggaris Logam 30 cm,200,4250,850000
5,Amplop Coklat A4,150,30000,4500000
6,Map Snelhekter Plastik,300,11000,3300000
7,Staples dan Isi,200,15000,3000000
8,Gunting Besar,150,50000,7500000
9,Korektor Pita,400,5100,2040000


## 4. Menyiapkan Database Barang dalam Bentuk Vektor dari Data Dummy 

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import openai 
from dotenv import load_dotenv
import os

CHROMA_PATH = "chroma"
DATA_PATH = "data"

os.environ['OPENAI_API_KEY'] = XXXXX
openai.api_key = XXXXXX

def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.txt")
    documents = loader.load()
    return documents


def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=750,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    #document = chunks[10]
    #print(document.page_content)
    #print(document.metadata)

    return chunks


def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

generate_data_store()

Split 91 documents into 91 chunks.
Saved 91 chunks to chroma.


/var/folders/md/k2hq3rtj2l91wlz0qjdkvttm0000gn/T/ipykernel_29958/4012481723.py:57: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


## 5. Mencari dan Mengeluarkan Barang yang Mirip dari Database dengan Vector Search

In [86]:
# Prepare the DB for retrieval.
embedding_function = OpenAIEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

#get sample 3 item
item_df_s = item_df.head(3).copy()

column_names = ['item_query','harga_query', 'nama_barang', 'jumlah', 'harga_satuan', 'sumber', 'score']

# Create an empty DataFrame with these columns
result_df = pd.DataFrame(columns=column_names)

for index,row in item_df_s.iterrows():
    results = db.similarity_search_with_relevance_scores(row['nama_barang'], 
                                                     k=3)#jumlah yang di-retrieve
    if(len(results) == 0 or results[0][1] < 0.6):
        print(f"Unable to find matching results.")
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

    for doc, _score in results:
        parsed_data = json.loads(doc.page_content.replace("'", "\""))
    # Convert data to DataFrame
        df_new = pd.DataFrame(parsed_data["data"])
        df_new['score'] = _score
        df_new['item_query'] = row['nama_barang']
        df_new['harga_query'] = row['harga_satuan']
        result_df = pd.concat([result_df, df_new], ignore_index=True)

result_df.rename(columns={'item_query':'barang_dokumen', 'harga_query':'harga_dokumen', 'nama_barang':'barang_database', 'harga_satuan':'harga_database', 'score':'skor_kemiripan'}, inplace=True)

result_df.drop('jumlah', axis=1, inplace=True)

result_df.head()


,barang_dokumen,harga_dokumen,barang_database,harga_database,sumber,skor_kemiripan
0,Pulpen Gel,6000,PENSIL FABER CASTELL PAKET UJIAN MANTAP,21000,padiumkm,0.647952
1,Pulpen Gel,6000,BALLPOINT GEL PEN JOYKO,3000,padiumkm,0.633821
2,Pulpen Gel,6000,PENSIL FABER CASTELL SET STANDAR,17000,padiumkm,0.629109
3,Pensil Mekanik,6000,"ISI PENSIL MEKANIK JOYKO PL-10 2B 2,0 X 90 MM",4000,padiumkm,0.701335
4,Pensil Mekanik,6000,"ISI PENSIL MEKANIK JOYKO PL-16 2B 2,0 X 120 MM",5000,padiumkm,0.700702


## 6. Melakukan Komparasi antara Barang di Dokumen dengan Harga Pasar

In [90]:
# Group by item_query and calculate average and standard deviation of harga_satuan
agg_df = result_df.groupby('barang_dokumen').agg(
    harga_dokumen=('harga_dokumen', 'first'),  # Get the first value of harga_query
    rerata_harga_database=('harga_database', 'mean'),  # Calculate mean of harga_satuan
    standar_deviasi_harga_database=('harga_database', 'std')     # Calculate standard deviation of harga_satuan
).reset_index()

# Ensure numeric types for harga_dokumen and rerata_harga_pasar
agg_df['harga_dokumen'] = pd.to_numeric(agg_df['harga_dokumen'], errors='coerce')
agg_df['rerata_harga_database'] = pd.to_numeric(agg_df['rerata_harga_database'], errors='coerce')

agg_df['perbedaan_harga'] = agg_df['harga_dokumen'] - agg_df['rerata_harga_database']
agg_df['persen_perbedaan_harga'] = agg_df['perbedaan_harga'] / agg_df['rerata_harga_database']

# Function to determine suspicion level
def suspicion_level(row):
    if row['persen_perbedaan_harga'] > 0.25:
        return 'Sangat Mencurigakan'
    elif 0.10 < row['persen_perbedaan_harga'] <= 0.25:
        return 'Sedikit Mencurigakan'
    else:
        return 'Tidak Mencurigakan'

# Apply function to DataFrame
agg_df['hasil_checking_ai'] = agg_df.apply(suspicion_level, axis=1)

# Convert DataFrame to JSON
json_result = agg_df.to_json(orient='records', lines=False)

# Display results
agg_df

,barang_dokumen,harga_dokumen,rerata_harga_database,standar_deviasi_harga_database,perbedaan_harga,persen_perbedaan_harga,hasil_checking_ai
0,Kertas HVS A4 (80gsm),41000,34666.666667,577.350269,6333.333333,0.182692,Sedikit Mencurigakan
1,Pensil Mekanik,6000,4166.666667,763.762616,1833.333333,0.440000,Sangat Mencurigakan
2,Pulpen Gel,6000,13666.666667,9451.631253,-7666.666667,-0.560976,Tidak Mencurigakan


## 7. Berinteraksi dengan Dokumen dan Hasil Analisa Trace.AI menggunakan Chatbot AI

In [70]:
# Define Chat Function
def model_generate_content(model: ChatSession, query: str, config):
    
    try:
        return model.send_message(query, generation_config=config).text
    except Exception as err:
        print(err)
        return "Sorry.. Service temporary unavailable."

In [91]:
# Start Chatting with the chatbot
model_chat = model_gemini.start_chat(response_validation=False)

generation_config = {
    "max_output_tokens": 8192,
    "temperature": 1,
    "top_p": 0.95,
}



start_template = f"""Anda adalah chatbot yang berfungsi untuk menjawab pertanyaan user terkait dokumen dan pengadaan barang berkaitan dengan sumber yang diberikan

Sumber Dokumen: {document_text}
List barang dan jasa: {response_item.text}

Dengan hasil checking sebagai berikut: {json_result}

"""

chat_1 = 'Ada barang jasa apa saja di dokumen ini?'

result_ai = model_generate_content(model_chat, start_template + chat_1, config=generation_config)

result_ai

'Dokumen ini berisi penawaran pengadaan berbagai jenis Alat Tulis Kantor (ATK).  Barang-barang yang ditawarkan meliputi:\n\n1. Pulpen Gel\n2. Pensil Mekanik\n3. Kertas HVS A4 (80gsm)\n4. Kertas HVS A3 (70gsm)\n5. Penggaris Logam 30 cm\n6. Amplop Coklat A4\n7. Map Snelhekter Plastik\n8. Staples dan Isi\n9. Gunting Besar\n10. Korektor Pita\n11. Tinta Stempel\n12. Tinta Printer\n13. Post-it Notes Warna-warni\n14. Klip Kertas Besar\n15. Pita Perekat Transparan\n16. Kertas Stiker Label\n17. Binder\n18. Buku Agenda\n19. Spidol Permanent\n20. Penghapus Papan Tulis\n21. Kalkulator Kantor\n22. Kertas Gambar\n23. Lem Kertas\n24. File Holder\n25. Tempat Sampah Mini\n26. Pisau Cutter\n27. Penghapus Pensil\n28. Papan Tulis Putih\n29. Jam Dinding Kantor\n30. Lembar Flipchart\n\n\nSelain barang-barang tersebut, dokumen juga mencantumkan rincian layanan tambahan berupa:\n\n* **Garansi Produk:** Garansi kualitas selama 6 bulan, termasuk penggantian produk rusak.\n* **Pengiriman Cepat dan Fleksibel:** E

Berdasarkan dokumen Proposal Penawaran Pengadaan Alat Tulis Kantor (ATK) dari PT. Alat Kantor Nusantara untuk Kementerian Keuangan, barang dan jasa yang ditawarkan meliputi:

## Barang:

- Pulpen Gel
- Pensil Mekanik
- Kertas HVS A4 (80gsm)
- Kertas HVS A3 (70gsm)
- Penggaris Logam 30 cm
- Amplop Coklat A4
- Map Snelhekter Plastik
- Staples dan Isi
- Gunting Besar
- Korektor Pita
- Tinta Stempel
- Tinta Printer
- Post-it Notes Warna-warni
- Klip Kertas Besar
- Pita Perekat Transparan
- Kertas Stiker Label
- Binder
- Buku Agenda
- Spidol Permanent
- Penghapus Papan Tulis
- Kalkulator Kantor
- Kertas Gambar
- Lem Kertas
- File Holder
- Tempat Sampah Mini
- Pisau Cutter
- Penghapus Pensil
- Papan Tulis Putih
- Jam Dinding Kantor
- Lembar Flipchart


In [74]:
# Ask Follow Up Questions to the chatbot
chat_2 = 'Apa barang yang terindikasi mencurigakan?'

result_ai = model_generate_content(model_chat, chat_2, config=generation_config)

result_ai

'Berdasarkan hasil *checking* yang diberikan, barang yang terindikasi mencurigakan adalah:\n\n* **Kertas HVS A4 (80gsm):**  Harga yang ditawarkan sedikit mencurigakan karena lebih tinggi dari rata-rata harga pasar.\n* **Pensil Mekanik:** Harga yang ditawarkan sangat mencurigakan karena jauh lebih tinggi dari rata-rata harga pasar.\n\nPerlu dicatat bahwa kesimpulan "mencurigakan" ini berdasarkan data eksternal (rata-rata harga pasar dan standar deviasi) yang tidak disertakan dalam dokumen proposal.  Hanya informasi hasil *checking* yang menjadi dasar penilaian ini.\n'

Berdasarkan hasil *checking* yang diberikan, barang yang terindikasi mencurigakan adalah:
* **Kertas HVS A4 (80gsm):**  Harga yang ditawarkan sedikit mencurigakan karena lebih tinggi dari rata-rata harga pasar.
* **Pensil Mekanik:** Harga yang ditawarkan sangat mencurigakan karena jauh lebih tinggi dari rata-rata harga pasar.

Perlu dicatat bahwa kesimpulan "mencurigakan" ini berdasarkan data eksternal (rata-rata harga pasar dan standar deviasi) yang tidak disertakan dalam dokumen proposal.  Hanya informasi hasil *checking* yang menjadi dasar penilaian ini.